# Most Polluted Countries - Limpieza de Datos

### · Antes de empezar... 

1. Para gestionar los Dataframes:

In [293]:
#Para mi doc de Pipeline:

import sys 
sys.path.append("../SRC") 
import support as sp 


In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from fuzzywuzzy import process, fuzz
import pickle

In [160]:
#Regex
import re

###  · Carga de CSVs

Empezamos cargando datos CSV con nuestros datos scrapeados de la natalidad en el periodo 2017/201

In [161]:
df_n17= pd.read_csv("../data/natalidad/natalidad2017.csv" ,  index_col = 0)
df_n17.head(5)

,0
1,"España [+] 2017 393.181 202.478 190.703 8,41‰ ..."
2,"Alemania [+] 2017 784.901 402.517 382.384 9,50..."
3,Reino Unido [+] 2017 754.754 387.030 367.724 1...
4,"Francia [+] 2017 770.045 394.058 375.987 11,50..."
5,"Italia [+] 2017 458.151 235.733 222.418 7,60‰ ..."


In [162]:
df_n18= pd.read_csv("../data/natalidad/natalidad2018.csv" ,  index_col = 0)
df_n18.head(5)

,0
1,"España [+] 2018 372.777 191.569 181.208 7,94‰ ..."
2,"Alemania [+] 2018 787.523 404.052 383.471 9,50..."
3,Reino Unido [+] 2018 730.918 375.379 355.539 1...
4,"Francia [+] 2018 759.199 388.944 370.255 11,30..."
5,"Italia [+] 2018 439.747 226.217 213.530 7,30‰ ..."


In [163]:
df_n19= pd.read_csv("../data/natalidad/natalidad2019.csv" ,  index_col = 0)
df_n19.head(5)

,0
1,"España [+] 2019 360.617 185.523 175.094 7,62‰ ..."
2,"Alemania [+] 2019 778.090 399.292 378.798 9,40..."
3,Reino Unido [+] 2019 712.699 375.379 355.539 1...
4,"Francia [+] 2019 754.008 385.340 368.668 11,20..."
5,"Italia [+] 2019 420.084 215.387 204.697 7,00‰ ..."


In [164]:
df_n20= pd.read_csv("../data/natalidad/natalidad2020.csv" ,  index_col = 0)
df_n20.head(5)

,0
1,"España [+] 2020 341.315 174.842 166.473 7,19‰ ..."
2,"Alemania [+] 2020 773.144 397.385 375.759 9,30..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2020 735.775 376.187 359.588 10,90..."
5,"Italia [+] 2020 404.892 208.431 196.461 6,80‰ ..."


In [165]:
df_n21= pd.read_csv("../data/natalidad/natalidad2021.csv" ,  index_col = 0)
df_n21.head(5)

,0
1,"España [+] 2021 337.380 174.148 163.232 7,12‰ ..."
2,"Alemania [+] 2021 795.492 397.385 375.759 9,60..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2021 738.594 376.187 359.588 10,90..."
5,"Italia [+] 2021 399.431 208.431 196.461 6,80‰ ..."


### · Limpieza Datos Scrapeados 

In [166]:
df_n17

,0
1,"España [+] 2017 393.181 202.478 190.703 8,41‰ ..."
2,"Alemania [+] 2017 784.901 402.517 382.384 9,50..."
3,Reino Unido [+] 2017 754.754 387.030 367.724 1...
4,"Francia [+] 2017 770.045 394.058 375.987 11,50..."
5,"Italia [+] 2017 458.151 235.733 222.418 7,60‰ ..."
...,...
189,"Samoa [+] 2017 24,69‰ 3,93 -1,26%"
190,"Yemen [+] 2017 31,04‰ 3,89 -2,60%"
191,"Sudáfrica [+] 2017 20,91‰ 2,43 -1,10%"
192,"Zambia [+] 2017 36,70‰ 4,72 -1,99%"


Empezaremos por limpiar el grueso de nuestros datos, discriminando los países. La decisión de hacerlo de esta manera es porque el split de la mayoría de información lo haremos por espacios y en los países no es válido ya que muchos tienen nombre compuesto.

In [167]:
df_n17= df_n17["0"].str.split("\s\[", regex= True, expand=True)
#df_n17

Para hacer la división del grueso de datos, crearemos un diccionario. Aprovecharemos que la estructura de los datos extraídos es fija para realizar el almacenamiento en el mismo. Lo primero es conocer la longitud de los diferentes paquetes de información, para crear los condicionales de los que dependerá este almacenamiento. En aquellos datos que no estén informados, con el objetivo de mantener la estructura, introduciremos un valor nulo. Este diccionario será nuestro puente para los DataFrames finales. 

In [168]:
lon=[]

for i in df_n17[1]:
    lon.append(len(i.split()))

In [169]:
lon[:5]

[8, 8, 8, 8, 8]

In [170]:
numeros_unicos = set(lon)

print(numeros_unicos)

{8, 5, 6, 7}


In [171]:
#lon.index(7)

Una vez sabemos la longitud de nuestros distintos paquetes de datos, empezamos con la creación del diccionario:

In [172]:
nat2017 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n17[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(str(a[3]))
        nat2017["female_born"].append(str(a[4]))
        nat2017["birth_rate"].append(str(a[5]))
        nat2017["fertility_rate"].append(str(a[6]))
        nat2017["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(str(a[3]))
        nat2017["female_born"].append(str(a[4]))
        nat2017["birth_rate"].append(str(a[5]))
        nat2017["fertility_rate"].append(str(a[6]))
        nat2017["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(np.nan)
        nat2017["female_born"].append(np.nan)
        nat2017["birth_rate"].append(str(a[3]))
        nat2017["fertility_rate"].append(str(a[4]))
        nat2017["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(np.nan)
        nat2017["male_born"].append(np.nan)
        nat2017["female_born"].append(np.nan)
        nat2017["birth_rate"].append(str(a[2]))
        nat2017["fertility_rate"].append(str(a[3]))
        nat2017["variation"].append(str(a[4]))
            

In [173]:
#control
len(nat2017["year"])

193

Una vez finalizado nuestro diccionario con el grueso de datos, debemos hacer la limpieza de la columna países. La limpieza se basará en eliminar espacios o "y" en los nombres, en su lugar pondremos "_": 

In [174]:
df_n17[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [175]:
df_n17[0] = df_n17[0].apply(lambda x: str(x).replace(" y ","_"))
df_n17[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [176]:
df_n17[0] = df_n17[0].apply(lambda x: str(x).replace(" ","_"))
df_n17[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

Ya disponemos de los países limpios por lo que están listos para ser unidos al resto del diccionario. Esto nos formará uno de nuestros cinco DataFrames finales. Haremos un diccionario con los países limpios, que después zipearemos con la información de nuestro anterior diccionario:

In [177]:
paises17={
        "country": []
            }

for i in df_n17[0]:
    
    paises17["country"].append(i)

In [178]:
#paises17

In [179]:
union17 = list(zip(paises17["country"], nat2017["year"], nat2017["born"], nat2017["male_born"], nat2017["female_born"], nat2017["birth_rate"], nat2017["fertility_rate"] , nat2017["variation"]))

###### *PUNTO DE CONTROL* 📍
Necesario para reseteo DataFrame

In [180]:
n17= pd.DataFrame(union17)
#n17

In [181]:
#control
#n17.info()
#n17.isnull().sum()

Nuestro siguiente paso será dar nombre a las columnas:

In [182]:
n17.columns

RangeIndex(start=0, stop=8, step=1)

In [183]:
n17.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/526978396.py:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  n17.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


El siguiente paso será gestionar los tipos de datos que tenemos. Nos interesa tener la mayor cantidad de datos numéricos posibles:

In [184]:
n17["year"]= n17["year"].astype("int")

In [185]:
n17["born"] = n17["born"].str.replace(".", "")

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/2687544233.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  n17["born"] = n17["born"].str.replace(".", "")


In [186]:
n17["born"] = pd.to_numeric(n17["born"])

In [187]:
n17["male_born"] = n17["male_born"].str.replace(".","")

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/2191722072.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  n17["male_born"] = n17["male_born"].str.replace(".","")


In [188]:
n17["male_born"] = pd.to_numeric(n17["male_born"])

In [189]:
n17["female_born"] = n17["female_born"].str.replace(".","")

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3584901649.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  n17["female_born"] = n17["female_born"].str.replace(".","")


In [190]:
n17["female_born"] = pd.to_numeric(n17["female_born"])

##### Gestión de porcentaje  

La gestión a realizar del porcentaje es su eliminación para evitar que sean objetos los números. Primero lo que haremos es sustituir las comas, por puntos, así nos permitirá tener un `float`. Acto seguido eliminados finalmente el porcentaje.  

In [191]:
n17["birth_rate"]= n17["birth_rate"].str.replace("‰","")

In [192]:
n17["birth_rate"]= n17["birth_rate"].str.replace(",","")

In [193]:
n17["birth_rate"] = pd.to_numeric(n17["birth_rate"])

In [194]:
n17["fertility_rate"] = n17["fertility_rate"].str.replace("%","")

In [195]:
n17["fertility_rate"] = n17["fertility_rate"].str.replace(",",".")

In [196]:
n17["fertility_rate"]= pd.to_numeric(n17["fertility_rate"])

In [197]:
n17["variation"] = n17["variation"].str.replace("%","")

In [198]:
n17["variation"]= n17["variation"].str.replace("‰","")

In [199]:
n17["variation"] = n17["variation"].str.replace(",",".")

In [200]:
n17["variation"]= pd.to_numeric(n17["variation"])

In [201]:
#n17

In [202]:
#control para saber como va mi transformación de datos
#n17.info()

#### ¡Lo tenemos! ✨

Primer DataFrame limpio y listo para ser concatenado. Vamos a por el resto.  

### 2018

Ahora a por los otros 4 años de información, donde repetiremos el proceso realizado. Para agilizar el proceso, crearemos funciones de limpieza que iremos aplicando a nuestra información:

In [203]:
df_n18

,0
1,"España [+] 2018 372.777 191.569 181.208 7,94‰ ..."
2,"Alemania [+] 2018 787.523 404.052 383.471 9,50..."
3,Reino Unido [+] 2018 730.918 375.379 355.539 1...
4,"Francia [+] 2018 759.199 388.944 370.255 11,30..."
5,"Italia [+] 2018 439.747 226.217 213.530 7,30‰ ..."
...,...
189,"Samoa [+] 2018 24,38‰ 3,88 -1,25%"
190,"Yemen [+] 2018 30,45‰ 3,79 -2,52%"
191,"Sudáfrica [+] 2018 20,51‰ 2,41 -1,03%"
192,"Zambia [+] 2018 36,19‰ 4,63 -1,80%"


In [204]:
df_n18= df_n18["0"].str.split("\s\[", regex= True, expand=True)
#df_n18

In [205]:
lon18=[]

for i in df_n18[1]:
    lon18.append(len(i.split()))

In [206]:
unicos18 = set(lon18)

print(unicos18)

{8, 5, 6, 7}


In [207]:
nat2018 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n18[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(str(a[3]))
        nat2018["female_born"].append(str(a[4]))
        nat2018["birth_rate"].append(str(a[5]))
        nat2018["fertility_rate"].append(str(a[6]))
        nat2018["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(str(a[3]))
        nat2018["female_born"].append(str(a[4]))
        nat2018["birth_rate"].append(str(a[5]))
        nat2018["fertility_rate"].append(str(a[6]))
        nat2018["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(np.nan)
        nat2018["female_born"].append(np.nan)
        nat2018["birth_rate"].append(str(a[3]))
        nat2018["fertility_rate"].append(str(a[4]))
        nat2018["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(np.nan)
        nat2018["male_born"].append(np.nan)
        nat2018["female_born"].append(np.nan)
        nat2018["birth_rate"].append(str(a[2]))
        nat2018["fertility_rate"].append(str(a[3]))
        nat2018["variation"].append(str(a[4]))

In [208]:
#control
len(nat2018["year"])

193

In [209]:
df_n18[0] = df_n18[0].apply(lambda x: str(x).replace(" y ","_"))
df_n18[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [210]:
df_n18[0] = df_n18[0].apply(lambda x: str(x).replace(" ","_"))
df_n18[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [211]:
paises18={
        "country": []
            }

for i in df_n18[0]:
    
    paises18["country"].append(i)

In [212]:
union18 = list(zip(paises18["country"], nat2018["year"], nat2018["born"], nat2018["male_born"], nat2018["female_born"], nat2018["birth_rate"], nat2018["fertility_rate"] , nat2018["variation"]))

#### *PUNTO DE CONTROL* 📍
Necesario para reseteo DataFrame

In [213]:
n18= pd.DataFrame(union18)
#n18

###### Limpieza datos
Nuestra primera función definida será para establecer las cabeceras de nuestras columnas:

In [214]:
def titulos_col(x):
    x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)

In [215]:
titulos_col(n18)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


Nuestras siguientes funciones a definir serán las que nos ayudaran a convertir nuestros objetos en números:

In [216]:
n18["year"]= n18["year"].astype("int")

In [217]:
def numeros(x):
    x = x.str.replace(".","")
    x = pd.to_numeric(x)
    return(x)

In [218]:
sp.numeros(n18["born"])

/Users/paulaelizagarate/Bootcamp/ETL_project/notebooks/../SRC/support.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


0      372777.0
1      787523.0
2      730918.0
3      759199.0
4      439747.0
         ...   
188         NaN
189         NaN
190         NaN
191         NaN
192         NaN
Name: born, Length: 193, dtype: float64

In [219]:
n18["born"] = numeros(n18["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [220]:
n18["male_born"]= numeros(n18["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [221]:
n18["female_born"]= numeros(n18["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


Por último haremos una función que limpie nuestros porcentajes, buscando de nuevo que queden en formato número para poder manejarlos:

In [222]:
def porcentajes (x):
    x = x.str.replace("‰","")
    x = x.str.replace("%","")
    x = x.str.replace(",",".")
    x = pd.to_numeric(x)
    return(x)

In [223]:
n18["birth_rate"]= porcentajes(n18["birth_rate"])

In [224]:
n18["fertility_rate"]= porcentajes(n18["fertility_rate"])

In [225]:
n18["variation"]= porcentajes(n18["variation"])

In [226]:
#n18

In [227]:
#control para saber como va la transformación de datos
#n18.info()

Ahora solo tendríamos que seguir aplicando nuestras funciones a nuestros DataFrames para terminar esta limpieza 🧽🪣  

### 2019

In [228]:
df_n19

,0
1,"España [+] 2019 360.617 185.523 175.094 7,62‰ ..."
2,"Alemania [+] 2019 778.090 399.292 378.798 9,40..."
3,Reino Unido [+] 2019 712.699 375.379 355.539 1...
4,"Francia [+] 2019 754.008 385.340 368.668 11,20..."
5,"Italia [+] 2019 420.084 215.387 204.697 7,00‰ ..."
...,...
189,"Samoa [+] 2019 24,10‰ 3,83 -1,21%"
190,"Yemen [+] 2019 29,87‰ 3,70 -2,43%"
191,"Sudáfrica [+] 2019 20,13‰ 2,38 -1,00%"
192,"Zambia [+] 2019 35,78‰ 4,56 -1,60%"


In [229]:
df_n19= df_n19["0"].str.split("\s\[", regex= True, expand=True)
#df_n19

In [230]:
lon19=[]

for i in df_n19[1]:
    lon19.append(len(i.split()))

In [231]:
unicos19 = set(lon19)

print(unicos19)

{8, 5, 6, 7}


In [232]:
nat2019 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n19[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(str(a[3]))
        nat2019["female_born"].append(str(a[4]))
        nat2019["birth_rate"].append(str(a[5]))
        nat2019["fertility_rate"].append(str(a[6]))
        nat2019["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(str(a[3]))
        nat2019["female_born"].append(str(a[4]))
        nat2019["birth_rate"].append(str(a[5]))
        nat2019["fertility_rate"].append(str(a[6]))
        nat2019["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(np.nan)
        nat2019["female_born"].append(np.nan)
        nat2019["birth_rate"].append(str(a[3]))
        nat2019["fertility_rate"].append(str(a[4]))
        nat2019["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(np.nan)
        nat2019["male_born"].append(np.nan)
        nat2019["female_born"].append(np.nan)
        nat2019["birth_rate"].append(str(a[2]))
        nat2019["fertility_rate"].append(str(a[3]))
        nat2019["variation"].append(str(a[4]))

In [233]:
#control
len(nat2019["year"])

193

In [234]:
df_n19[0] = df_n19[0].apply(lambda x: str(x).replace(" y ","_"))
df_n19[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [235]:
df_n19[0] = df_n19[0].apply(lambda x: str(x).replace(" ","_"))
df_n19[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [236]:
paises19={
        "country": []
            }

for i in df_n19[0]:
    
    paises19["country"].append(i)

In [237]:
union19 = list(zip(paises19["country"], nat2019["year"], nat2019["born"], nat2019["male_born"], nat2019["female_born"], nat2019["birth_rate"], nat2019["fertility_rate"] , nat2019["variation"]))

In [238]:
n19= pd.DataFrame(union19)
#n19

##### Limpiamos Datos 

In [239]:
titulos_col(n19)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


In [240]:
n19["year"]= n19["year"].astype("int")

In [241]:
n19["born"]= numeros(n19["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [242]:
n19["male_born"]= numeros(n19["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [243]:
n19["female_born"]= numeros(n19["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [244]:
n19["birth_rate"] = porcentajes(n19["birth_rate"])

In [245]:
n19["fertility_rate"] = porcentajes(n19["fertility_rate"])

In [246]:
n19["variation"] = porcentajes(n19["variation"])

In [247]:
n19

,country,year,born,male_born,female_born,birth_rate,fertility_rate,variation
0,España,2019,360617.0,185523.0,175094.0,7.62,1.24,-2.24
1,Alemania,2019,778090.0,399292.0,378798.0,9.40,1.54,-1.91
2,Reino_Unido,2019,712699.0,375379.0,355539.0,10.70,1.63,-2.98
3,Francia,2019,754008.0,385340.0,368668.0,11.20,1.86,-0.53
4,Italia,2019,420084.0,215387.0,204697.0,7.00,1.27,-1.55
...,...,...,...,...,...,...,...,...
188,Samoa,2019,NaN,NaN,NaN,24.10,3.83,-1.21
189,Yemen,2019,NaN,NaN,NaN,29.87,3.70,-2.43
190,Sudáfrica,2019,NaN,NaN,NaN,20.13,2.38,-1.00
191,Zambia,2019,NaN,NaN,NaN,35.78,4.56,-1.60


In [248]:
#control de como va mi transformación de datos:
#n19.info()

### 2020

In [249]:
df_n20

,0
1,"España [+] 2020 341.315 174.842 166.473 7,19‰ ..."
2,"Alemania [+] 2020 773.144 397.385 375.759 9,30..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2020 735.775 376.187 359.588 10,90..."
5,"Italia [+] 2020 404.892 208.431 196.461 6,80‰ ..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [250]:
df_n20= df_n20["0"].str.split("\s\[", regex= True, expand=True)
df_n20

,0,1
1,España,"+] 2020 341.315 174.842 166.473 7,19‰ 1,19 -4,03%"
2,Alemania,"+] 2020 773.144 397.385 375.759 9,30‰ 1,53 -0,65%"
3,Reino Unido,"+] 2020 712.699 375.379 355.539 10,20‰ 1,56 -4..."
4,Francia,"+] 2020 735.775 376.187 359.588 10,90‰ 1,83 -1..."
5,Italia,"+] 2020 404.892 208.431 196.461 6,80‰ 1,24 -2,36%"
...,...,...
189,Samoa,"+] 2020 23,86‰ 3,79 -1,17%"
190,Yemen,"+] 2020 29,30‰ 3,61 -2,32%"
191,Sudáfrica,"+] 2020 19,77‰ 2,36 -0,97%"
192,Zambia,"+] 2020 35,44‰ 4,50 -1,38%"


In [251]:
lon20=[]

for i in df_n20[1]:
    lon20.append(len(i.split()))

In [252]:
unicos20 = set(lon20)

print(unicos20)

{8, 5, 6, 7}


In [253]:
nat2020 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n20[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(str(a[3]))
        nat2020["female_born"].append(str(a[4]))
        nat2020["birth_rate"].append(str(a[5]))
        nat2020["fertility_rate"].append(str(a[6]))
        nat2020["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(str(a[3]))
        nat2020["female_born"].append(str(a[4]))
        nat2020["birth_rate"].append(str(a[5]))
        nat2020["fertility_rate"].append(str(a[6]))
        nat2020["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(np.nan)
        nat2020["female_born"].append(np.nan)
        nat2020["birth_rate"].append(str(a[3]))
        nat2020["fertility_rate"].append(str(a[4]))
        nat2020["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(np.nan)
        nat2020["male_born"].append(np.nan)
        nat2020["female_born"].append(np.nan)
        nat2020["birth_rate"].append(str(a[2]))
        nat2020["fertility_rate"].append(str(a[3]))
        nat2020["variation"].append(str(a[4]))

In [254]:
#control
len(nat2020["year"])

193

In [255]:
df_n20[0] = df_n20[0].apply(lambda x: str(x).replace(" y ","_"))
df_n20[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [256]:
df_n20[0] = df_n20[0].apply(lambda x: str(x).replace(" ","_"))
df_n20[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [257]:
paises20={
        "country": []
            }

for i in df_n20[0]:
    
    paises20["country"].append(i)

In [258]:
union20 = list(zip(paises20["country"], nat2020["year"], nat2020["born"], nat2020["male_born"], nat2020["female_born"], nat2020["birth_rate"], nat2020["fertility_rate"] , nat2020["variation"]))

In [259]:
n20= pd.DataFrame(union20)
n20

,0,1,2,3,4,5,6,7
0,España,2020,341.315,174.842,166.473,"7,19‰","1,19","-4,03%"
1,Alemania,2020,773.144,397.385,375.759,"9,30‰","1,53","-0,65%"
2,Reino_Unido,2020,712.699,375.379,355.539,"10,20‰","1,56","-4,29%"
3,Francia,2020,735.775,376.187,359.588,"10,90‰","1,83","-1,61%"
4,Italia,2020,404.892,208.431,196.461,"6,80‰","1,24","-2,36%"
...,...,...,...,...,...,...,...,...
188,Samoa,2020,NaN,NaN,NaN,"23,86‰","3,79","-1,17%"
189,Yemen,2020,NaN,NaN,NaN,"29,30‰","3,61","-2,32%"
190,Sudáfrica,2020,NaN,NaN,NaN,"19,77‰","2,36","-0,97%"
191,Zambia,2020,NaN,NaN,NaN,"35,44‰","4,50","-1,38%"


##### *Limpiamos Datos* 

In [260]:
titulos_col(n20)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


In [261]:
n20["year"]= n20["year"].astype("int")

In [262]:
n20["born"] = numeros(n20["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [263]:
n20["male_born"]= numeros(n20["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [264]:
n20["female_born"]= numeros(n20["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [265]:
n20["birth_rate"]= porcentajes(n20["birth_rate"])

In [266]:
n20["fertility_rate"]= porcentajes(n20["fertility_rate"])

In [267]:
n20["variation"]= porcentajes(n20["variation"])

In [268]:
#n20

In [269]:
#control transformación datos
#n20.info()

### 2021

In [270]:
df_n21

,0
1,"España [+] 2021 337.380 174.148 163.232 7,12‰ ..."
2,"Alemania [+] 2021 795.492 397.385 375.759 9,60..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2021 738.594 376.187 359.588 10,90..."
5,"Italia [+] 2021 399.431 208.431 196.461 6,80‰ ..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [271]:
df_n21= df_n21["0"].str.split("\s\[", regex= True, expand=True)
#df_n21

In [272]:
lon21=[]

for i in df_n21[1]:
    lon21.append(len(i.split()))

In [273]:
unicos21 = set(lon21)

print(unicos21)

{8, 5, 6, 7}


In [274]:
nat2021 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n21[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(str(a[3]))
        nat2021["female_born"].append(str(a[4]))
        nat2021["birth_rate"].append(str(a[5]))
        nat2021["fertility_rate"].append(str(a[6]))
        nat2021["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(str(a[3]))
        nat2021["female_born"].append(str(a[4]))
        nat2021["birth_rate"].append(str(a[5]))
        nat2021["fertility_rate"].append(str(a[6]))
        nat2021["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(np.nan)
        nat2021["female_born"].append(np.nan)
        nat2021["birth_rate"].append(str(a[3]))
        nat2021["fertility_rate"].append(str(a[4]))
        nat2021["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(np.nan)
        nat2021["male_born"].append(np.nan)
        nat2021["female_born"].append(np.nan)
        nat2021["birth_rate"].append(str(a[2]))
        nat2021["fertility_rate"].append(str(a[3]))
        nat2021["variation"].append(str(a[4]))

In [275]:
#control
len(nat2021["year"])

193

In [276]:
df_n21[0] = df_n21[0].apply(lambda x: str(x).replace(" y ","_"))
df_n21[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [277]:
df_n21[0] = df_n21[0].apply(lambda x: str(x).replace(" ","_"))
df_n21[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [278]:
paises21={
        "country": []
            }

for i in df_n21[0]:
    
    paises21["country"].append(i)

In [279]:
union21 = list(zip(paises21["country"], nat2021["year"], nat2021["born"], nat2021["male_born"], nat2021["female_born"], nat2021["birth_rate"], nat2021["fertility_rate"] , nat2021["variation"]))

In [280]:
n21= pd.DataFrame(union21)
#n21

##### Limpiamos Datos 

In [281]:
titulos_col(n21)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


In [282]:
n21["year"]= n21["year"].astype("int")

In [283]:
n21["born"] = numeros(n21["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [284]:
n21["male_born"]= numeros(n21["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [285]:
n21["female_born"]= numeros(n21["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1171/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [286]:
n21["birth_rate"]= porcentajes(n21["birth_rate"])

In [287]:
n21["fertility_rate"]= porcentajes(n21["fertility_rate"])

In [288]:
n21["variation"]= porcentajes(n21["variation"])

In [289]:
#n21

In [290]:
#control para saber como llevo la transformación de datos
#n21.info()

### Concatenación Tablas Natalidad 

Como punto final, concatenaremos las cinco tablas que hemos limpiado para formar una única tabla con los datos de natalidad comprendidos entre 2017/2021:

In [291]:
natalidad_total= pd.concat([n17, n18, n19, n20, n21], axis=0, ignore_index = True)

In [292]:
natalidad_total

,country,year,born,male_born,female_born,birth_rate,fertility_rate,variation
0,España,2017,393181.0,202478.0,190703.0,841.00,1.31,-1.92
1,Alemania,2017,784901.0,402517.0,382384.0,950.00,1.57,-1.88
2,Reino_Unido,2017,754754.0,387030.0,367724.0,1140.00,1.74,-2.79
3,Francia,2017,770045.0,394058.0,375987.0,1150.00,1.89,-1.56
4,Italia,2017,458151.0,235733.0,222418.0,760.00,1.32,-1.49
...,...,...,...,...,...,...,...,...
960,Samoa,2020,NaN,NaN,NaN,23.86,3.79,-1.17
961,Yemen,2020,NaN,NaN,NaN,29.30,3.61,-2.32
962,Sudáfrica,2020,NaN,NaN,NaN,19.77,2.36,-0.97
963,Zambia,2020,NaN,NaN,NaN,35.44,4.50,-1.38


In [137]:
#control todos los registros en tabla final
lentotal= len(n21) + len(n20) + len(n19) + len(n18) + len(n17)
lentotal

965

In [294]:
natalidad_total.fillna("0", inplace= True)
natalidad_total.head(10)

,country,year,born,male_born,female_born,birth_rate,fertility_rate,variation
0,España,2017,393181.0,202478.0,190703.0,841.0,1.31,-1.92
1,Alemania,2017,784901.0,402517.0,382384.0,950.0,1.57,-1.88
2,Reino_Unido,2017,754754.0,387030.0,367724.0,1140.0,1.74,-2.79
3,Francia,2017,770045.0,394058.0,375987.0,1150.0,1.89,-1.56
4,Italia,2017,458151.0,235733.0,222418.0,760.0,1.32,-1.49
5,Portugal,2017,86154.0,44072.0,42082.0,840.0,1.38,1.47
6,Estados_Unidos,2017,0,0,0,1180.0,1.77,-3.02
7,Japón,2017,0,0,0,760.0,1.43,-0.69
8,China,2017,0,0,0,1264.0,1.68,0.48
9,Andorra,2016,634.0,364.0,373.0,880.0,1.27,6.72


In [295]:
natalidad_total["variation"]= pd.to_numeric(natalidad_total["variation"])

### Traducción Países DataFrame

In [138]:
pip install deep_translator

Note: you may need to restart the kernel to use updated packages.


In [296]:
from deep_translator import GoogleTranslator

In [297]:
traductor= GoogleTranslator(source='es', target='en')

In [298]:
country= list(natalidad_total["country"])

In [299]:
resultado = traductor.translate_batch(country)

In [300]:
natalidad_total["country"]= resultado

Volvemos a trabajar el formato de la columna países ya que al traducir lo hemos perdido.  

In [301]:
natalidad_total["country"]= natalidad_total["country"].apply(lambda x: str(x).replace(" ","_"))

In [302]:
natalidad_total["country"].unique()

array(['Spain', 'Germany', 'United_Kingdom', 'France', 'Italy',
       'Portugal', 'USA', 'Japan', 'China', 'Andorra',
       'United_Arab_Emirates', 'Afghanistan', 'Antigua_Barbuda',
       'Albanian', 'Armenia', 'Angola', 'Argentina', 'Austria',
       'Australia', 'Azerbaijan', 'Bosnia_Herzegovina', 'barbados',
       'Bangladesh', 'Belgium', 'Burkina_Faso', 'Bulgaria', 'bahrain',
       'Burundi', 'Benin', 'Brunei', 'bolivian', 'Brazil', 'Bahamas',
       'bhutan', 'botswana', 'belarus', 'belize', 'Canada',
       'Democratic_Republic_of_Congo', 'Central_African_Republic',
       'Republic_of_Congo', 'Swiss', 'Ivory_Coast', 'Chili', 'Cameroon',
       'Colombia', 'Costa_Rica', 'Cuba', 'Cape_Verde', 'Cyprus',
       'czech_republic', 'djibouti', 'Denmark', 'Dominica',
       'Dominican_Republic', 'Algeria', 'Ecuador', 'Estonia', 'Egypt',
       'eritrea', 'Ethiopia', 'Finland', 'fiji',
       'Federated_States_of_Micronesia', 'Gabon', 'Grenade', 'Georgia',
       'Ghana', 'Gambia', 

In [303]:
def minusculas (x):
    x= [i.lower() for i in x]
    return(x)

In [304]:
natalidad_total["country"]= minusculas(natalidad_total["country"])

In [305]:
#natalidad_total.head(5)

In [306]:
natalidad_total["country"]= natalidad_total["country"].apply(lambda x: str(x).replace("_"," "))
natalidad_total["country"]

0               spain
1             germany
2      united kingdom
3              france
4               italy
            ...      
960             samoa
961             yemen
962      south africa
963            zambia
964          zimbabwe
Name: country, Length: 965, dtype: object

In [307]:
natalidad_total["country"]= natalidad_total["country"].apply(lambda x: str(x).title())

Ahora si que si, tenemos nuestros datos de natalidad listos para ser utilizados. Los guardaremos para utilizarlos más adelante.  

In [308]:
natalidad_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965 entries, 0 to 964
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         965 non-null    object 
 1   year            965 non-null    int64  
 2   born            965 non-null    object 
 3   male_born       965 non-null    object 
 4   female_born     965 non-null    object 
 5   birth_rate      965 non-null    float64
 6   fertility_rate  965 non-null    float64
 7   variation       965 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 60.4+ KB


In [309]:
natalidad_total.to_csv('birthrate.csv')